In [ ]:
import sys
sys.path.append('../..')
from src import WindSpeedUnit, ForecastAPI, ForecastCurrent, HistoricalAPI, HistoricalDaily, HistoricalHourly
import src
import pandas as pd
import datetime as dt

f = HistoricalAPI()
f.latitude, f.longitude = (53.98352, -6.39139) #src.GEO_COORDINATES.DUNDALK_IT.value
f.wind_speed_unit = WindSpeedUnit.METERS_PER_SECOND
# f.daily = [HistoricalDaily.TEMPERATURE_2M_MEAN, HistoricalDaily.WIND_SPEED_10M_MAX]
# f.daily = list(HistoricalDaily._member_map_.values())
# f.hourly = [HistoricalHourly.TEMPERATURE_2M, HistoricalHourly.WIND_SPEED_10M]
# f.hourly = list(HistoricalHourly._member_map_.values())
f.hourly = [HistoricalHourly.TEMPERATURE_2M, HistoricalHourly.RELATIVE_HUMIDITY_2M, HistoricalHourly.PRECIPITATION, HistoricalHourly.WIND_SPEED_10M,
            HistoricalHourly.WIND_DIRECTION_10M, HistoricalHourly.WIND_SPEED_100M, HistoricalHourly.WIND_DIRECTION_100M]
f.start_date = dt.datetime(2010, 1, 1)
f.end_date = dt.datetime(2024, 12, 31)
f.cell_selection = src.enums.CellSelection.NEAREST

response = f.request()
print(f.build_url())
print(response)

reindexing tables


In [5]:
url=f.build_url()
conn = f._conn
src.weather.utils.ApiCounter.calculate_call_weight_from_url(url) 

273.9

In [6]:
parsedParams = src.weather.utils.ApiCounter.parse_url_params(url)
startDate = dt.datetime.strptime(parsedParams['start_date'], '%Y-%m-%d')#.strftime('%Y-%m-%d %H:%M:%S')
endDate = dt.datetime.strptime(parsedParams['end_date'], '%Y-%m-%d')#.strftime('%Y-%m-%d %H:%M:%S')
parsedParams['start_date'] = startDate.strftime('%Y-%m-%d %H:%M:%S')
parsedParams['end_date'] = endDate.strftime('%Y-%m-%d %H:%M:%S')

In [7]:
pd.read_sql_query(f'''
                  SELECT * FROM hourly_historical_weather_data
                  WHERE time BETWEEN ? AND ?
                  AND {'1=0' if parsedParams.get('hourly') is None else 'parameter IN (' + ','.join("'"+str(x)+"'" for x in parsedParams['hourly']) + ')'}
                  AND latitude = ?
                  AND longitude = ?
                  ''', conn, params=[parsedParams['start_date'], parsedParams['end_date'], parsedParams['latitude'][0], parsedParams['longitude'][0]])

,ID,time,latitude,longitude,utc_offset_seconds,timezone,timezone_abbreviation,elevation,parameter,value
0,94657,2010-01-01 00:00:00,53.98352,-6.39139,0,GMT,GMT,15.0,precipitation,0.0
1,313081,2010-01-01 00:00:00,53.98352,-6.39139,0,GMT,GMT,15.0,relative_humidity_2m,85.0
2,531505,2010-01-01 00:00:00,53.98352,-6.39139,0,GMT,GMT,15.0,temperature_2m,-0.4
3,749929,2010-01-01 00:00:00,53.98352,-6.39139,0,GMT,GMT,15.0,wind_direction_100m,28.0
4,968353,2010-01-01 00:00:00,53.98352,-6.39139,0,GMT,GMT,15.0,wind_direction_10m,12.0
...,...,...,...,...,...,...,...,...,...,...
920306,3985,2024-12-31 00:00:00,53.98352,-6.39139,0,GMT,GMT,15.0,temperature_2m,9.1
920307,4729,2024-12-31 00:00:00,53.98352,-6.39139,0,GMT,GMT,15.0,wind_direction_100m,220.0
920308,5473,2024-12-31 00:00:00,53.98352,-6.39139,0,GMT,GMT,15.0,wind_direction_10m,219.0
920309,6217,2024-12-31 00:00:00,53.98352,-6.39139,0,GMT,GMT,15.0,wind_speed_100m,43.6
